In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
import http.client
import ssl
import gzip
from io import BytesIO
import json
import pandas as pd
import re
import openpyxl

In [2]:
firefox_options = Options()
firefox_options.add_argument('--headless')

driver = webdriver.Firefox(options=firefox_options)

In [3]:
archivo_excel = 'links_d1.xlsx'
df = pd.read_excel(archivo_excel)

columna_links = df['LINK']

links = columna_links.tolist()

In [4]:
stores = {"Bogotá":"12165","Medellin":"12310","Barranquilla":"12341","Bucaramanga":"12437","Cali":"12490"}

In [23]:
def fetch_sku(url):
    match = re.search(r'(\d+)$', url)
    
    if match:
        number = match.group(1)
        return number
    else:
        return 0 
    
def fetch_product(sku, store):
    conn = http.client.HTTPSConnection("nextgentheadless.instaleap.io", context=ssl._create_unverified_context() )
    
    payload = json.dumps([
      {
        "operationName": "GetProductsBySKU",
        "variables": {
          "getProductsBySkuInput": {
            "clientId": "D1",
            "skus": [
              sku
            ],
            "storeReference": store
          }
        },
        "query": "fragment CategoryFields on CategoryModel {\n  active\n  boost\n  hasChildren\n  categoryNamesPath\n  isAvailableInHome\n  level\n  name\n  path\n  reference\n  slug\n  photoUrl\n  imageUrl\n  shortName\n  isFeatured\n  isAssociatedToCatalog\n  __typename\n}\n\nfragment CatalogProductTagModel on CatalogProductTagModel {\n  description\n  enabled\n  textColor\n  filter\n  tagReference\n  backgroundColor\n  name\n  __typename\n}\n\nfragment CatalogProductFormatModel on CatalogProductFormatModel {\n  format\n  equivalence\n  unitEquivalence\n  clickMultiplier\n  minQty\n  maxQty\n  __typename\n}\n\nfragment Taxes on ProductTaxModel {\n  taxId\n  taxName\n  taxType\n  taxValue\n  taxSubTotal\n  __typename\n}\n\nfragment PromotionCondition on PromotionCondition {\n  quantity\n  price\n  priceBeforeTaxes\n  taxTotal\n  taxes {\n    ...Taxes\n    __typename\n  }\n  __typename\n}\n\nfragment Promotion on Promotion {\n  type\n  isActive\n  conditions {\n    ...PromotionCondition\n    __typename\n  }\n  description\n  endDateTime\n  startDateTime\n  __typename\n}\n\nfragment PromotedModel on PromotedModel {\n  isPromoted\n  onLoadBeacon\n  onClickBeacon\n  onViewBeacon\n  onBasketChangeBeacon\n  onWishlistBeacon\n  __typename\n}\n\nfragment SpecificationModel on SpecificationModel {\n  title\n  values {\n    label\n    value\n    __typename\n  }\n  __typename\n}\n\nfragment NutritionalDetailsInformation on NutritionalDetailsInformation {\n  servingName\n  servingSize\n  servingUnit\n  servingsPerPortion\n  nutritionalTable {\n    nutrientName\n    quantity\n    unit\n    quantityPerPortion\n    dailyValue\n    __typename\n  }\n  bottomInfo\n  __typename\n}\n\nfragment CatalogProductModel on CatalogProductModel {\n  name\n  price\n  photosUrl\n  unit\n  subUnit\n  subQty\n  description\n  sku\n  ean\n  maxQty\n  minQty\n  clickMultiplier\n  nutritionalDetails\n  isActive\n  slug\n  brand\n  stock\n  securityStock\n  boost\n  isAvailable\n  location\n  priceBeforeTaxes\n  taxTotal\n  promotion {\n    ...Promotion\n    __typename\n  }\n  taxes {\n    ...Taxes\n    __typename\n  }\n  categories {\n    ...CategoryFields\n    __typename\n  }\n  categoriesData {\n    ...CategoryFields\n    __typename\n  }\n  formats {\n    ...CatalogProductFormatModel\n    __typename\n  }\n  tags {\n    ...CatalogProductTagModel\n    __typename\n  }\n  specifications {\n    ...SpecificationModel\n    __typename\n  }\n  promoted {\n    ...PromotedModel\n    __typename\n  }\n  score\n  relatedProducts\n  ingredients\n  stockWarning\n  nutritionalDetailsInformation {\n    ...NutritionalDetailsInformation\n    __typename\n  }\n  productVariants\n  isVariant\n  isDominant\n  __typename\n}\n\nquery GetProductsBySKU($getProductsBySkuInput: GetProductsBySKUInput!) {\n  getProductsBySKU(getProductsBySKUInput: $getProductsBySkuInput) {\n    ...CatalogProductModel\n    __typename\n  }\n}"
      }
    ])
    headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:129.0) Gecko/20100101 Firefox/129.0',
      'Accept': '*/*',
      'Accept-Language': 'en-US,en;q=0.5',
      'Accept-Encoding': 'gzip, deflate, br, zstd',
      'content-type': 'application/json',
      'apollographql-client-name': 'Ecommerce',
      'apollographql-client-version': '3.50.3',
      'token': '',
      'dpl-api-key': '',
      'sentry-trace': '608e3f3b7a8a47d99e05eddfafb2410c-a44a86329ba7dcaa-1',
      'baggage': 'sentry-environment=production,sentry-release=3.50.3,sentry-public_key=b29a7fe3cf5f486b9520c5b3de53a5a7,sentry-trace_id=608e3f3b7a8a47d99e05eddfafb2410c,sentry-sample_rate=0.1,sentry-sampled=true',
      'Origin': 'https://domicilios.tiendasd1.com',
      'Connection': 'keep-alive',
      'Referer': 'https://domicilios.tiendasd1.com/',
      'Sec-Fetch-Dest': 'empty',
      'Sec-Fetch-Mode': 'cors',
      'Sec-Fetch-Site': 'cross-site',
      'Priority': 'u=4',
      'TE': 'trailers'
    }



    conn.request("POST", "/api/v3", payload, headers)
    
    res = conn.getresponse()
    data = res.read()
    if res.getheader('Content-Encoding') == 'gzip':
        data = gzip.GzipFile(fileobj=BytesIO(data)).read()

    data = json.loads(data.decode('utf-8'))
    
    conn.close()

    return data

def fetch_products(urls,store):
    links = []
    names = []
    prices_without_discount = []
    
    for url in urls:
        try:
            data = fetch_product(fetch_sku(url),store)

            product = data[0]['data']['getProductsBySKU'][0]
                
            links.extend([url])
            names.extend([product['name']])
            prices_without_discount.extend([product['priceBeforeTaxes']])
        except:
            links.extend([url])
            names.extend(['no disponible'])
            prices_without_discount.extend(['no disponible'])
            print(f"error en {url}")

    df = pd.DataFrame({
            'Link': links,
            'Name': names,
            'Regular Price': prices_without_discount,
            'Promotional Price': prices_without_discount
    })
    
    return df


In [26]:
df = fetch_products(links, stores['Bogotá'])

error en https://domicilios.tiendasd1.com/p/barra-de-chocolate-con-mani-jumbo-40-g-12001688
error en https://domicilios.tiendasd1.com/p/zucaritas-kelloggs-200-g-12002711
error en https://domicilios.tiendasd1.com/p/chocolatina-jet-x-10-u-12004888
error en https://domicilios.tiendasd1.com/p/salchicha-tradicional-viande-450-grs-12000082
error en https://domicilios.tiendasd1.com/p/salsa-bolonesa-deliziare-400-g-12000451
error en https://domicilios.tiendasd1.com/p/crema-de-leche-larga-vida-latti-200-ml-12001250
error en https://domicilios.tiendasd1.com/p/doritos-familiar-175-grs-12002003
error en https://domicilios.tiendasd1.com/p/refrescos-surtidos-frutas-6-und-12000297 
error en https://domicilios.tiendasd1.com/p/jabon-de-manos-liquido-natural-feeling-500-ml-12000163
error en https://domicilios.tiendasd1.com/p/copitos-little-angels-100-und-12000964
error en https://domicilios.tiendasd1.com/p/salchicha-perro-americana-big-bob-430-g-12001607
error en https://domicilios.tiendasd1.com/p/galle

In [27]:
df.to_excel('productos_d1_links.xlsx', index=False)